# Road Safety in San Diego County

In [1]:
!pip install geojson

In [2]:
#imports 

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import string

# Display plots directly in the notebook instead of in a new window
%matplotlib inline

In [3]:
# Configure libraries
# The seaborn library makes plots look nicer
sns.set()
sns.set_context('talk')

# Round decimals when displaying DataFrames
pd.set_option('precision', 2)

## Data frame management

We should figure out how we want to structure the data frames. 
Options include: 
- Merge into one df
- Find matching identifiers and try to merge data by these
- Work in completely independent dfs

In [18]:
df_police_stops = pd.read_csv("datasets/police_stops/ripa_stops_datasd_v1.csv")
df_collisions = pd.read_csv("datasets/traffic_collisions/pd_collisions_datasd_v1.csv")
df_police_calls1 = pd.read_csv("datasets/police_calls/pd_calls_for_service_2015_datasd_v1.csv")
df_police_calls2 = pd.read_csv("datasets/police_calls/pd_calls_for_service_2016_datasd_v1.csv")
df_police_calls3 = pd.read_csv("datasets/police_calls/pd_calls_for_service_2017_datasd_v1.csv")
df_police_calls4 = pd.read_csv("datasets/police_calls/pd_calls_for_service_2018_datasd.csv")
df_police_calls5 = pd.read_csv("datasets/police_calls/pd_calls_for_service_2019_datasd.csv")

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,9,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
# Merging all of the df_police_calls databases
df_police_calls = pd.merge(df_police_calls1,df_police_calls2,df_police_calls3, df_police_calls4, df_police_calls5,on='stop_id')

TypeError: merge() got multiple values for argument 'on'

In [16]:
print(df_police_calls)

                  0                    1   2      3    4                 5   \
0       P15010000001  2015-01-01 00:00:14   3   3800  NaN             DELTA   
1       P15010000002  2015-01-01 00:00:30   3      0  NaN              60TH   
2       P15010000003  2015-01-01 00:00:48   3   3600  NaN             LOGAN   
3       P15010000004  2015-01-01 00:00:57   3      0  NaN              33RD   
4       P15010000005  2015-01-01 00:01:05   3   3300  NaN              43RD   
5       P15010000006  2015-01-01 00:01:27   3   1400  NaN         HORNBLEND   
6       P15010000007  2015-01-01 00:01:35   3    700  NaN              05TH   
7       P15010000008  2015-01-01 00:01:37   3    700  NaN         OLIVEWOOD   
8       P15010000009  2015-01-01 00:01:39   3   2600  NaN           BAYSIDE   
9       P15010000010  2015-01-01 00:02:13   3   4000  NaN            EUCLID   
10      P15010000011  2015-01-01 00:02:18   3   2600  NaN         ROSECRANS   
11      P15010000012  2015-01-01 00:02:41   3   3200

# DATA CLEANING

## df_collisions

In [6]:
# Splitting datetime for the datasets

# Splitting datetime for df_collisions
# Creating seperate columns for Date and Time
date_time = df_collisions['date_time'].str.split(' ', n=1, expand=True)
df_collisions = df_collisions.drop(['date_time'], axis=1)
df_collisions['date'] = date_time[0]
df_collisions['time'] = date_time[1]

In [7]:
# droping useless columns
drop_cols_collision = ['address_pd_primary','address_pd_intersecting', 'address_name_intersecting', 'address_sfx_intersecting','report_id']
df_collisions = df_collisions.drop(drop_cols_collision, axis = 1)


In [8]:
# Making everything lowercase

df_collisions['address_road_primary'] = df_collisions['address_road_primary'].str.lower()
df_collisions['address_sfx_primary'] = df_collisions['address_sfx_primary'].str.lower()
df_collisions['charge_desc'] = df_collisions['charge_desc'].str.lower()
df_collisions['hit_run_lvl'] = df_collisions['hit_run_lvl'].str.lower()

In [9]:
#stripping gets rid of extraneous spaces to allow for easier processing of data
df_collisions['address_road_primary'] = df_collisions.address_road_primary.str.strip()
df_collisions['address_sfx_primary'] = df_collisions.address_sfx_primary.str.strip()
df_collisions['violation_section']= df_collisions.violation_section.str.strip()
df_collisions['violation_type'] = df_collisions.violation_type.str.strip()
df_collisions['charge_desc'] = df_collisions.charge_desc.str.strip()
df_collisions['hit_run_lvl'] = df_collisions.hit_run_lvl.str.strip()

In [10]:
# convert the 'Date' column to datetime format 
df_collisions['date']= pd.to_datetime(df_collisions['date']) 
df_collisions['time']= pd.to_datetime(df_collisions['time']) 

In [11]:
df_collisions.dtypes

police_beat                        int64
address_number_primary             int64
address_road_primary              object
address_sfx_primary               object
violation_section                 object
violation_type                    object
charge_desc                       object
injured                            int64
killed                             int64
hit_run_lvl                       object
date                      datetime64[ns]
time                      datetime64[ns]
dtype: object

## df_police calls

In [12]:
# Change entries in col 'day_of_week' to represent the day of the week in words
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(1,'Monday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(2,'Tuesday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(3,'Wednesday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(4,'Thursday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(5,'Friday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(6,'Saturday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(7,'Sunday')

In [13]:
# Splitting datetime for df_police_calls
date_time = df_police_calls['date_time'].str.split(' ', n=1, expand=True)
df_police_calls = df_police_calls.drop(['date_time'], axis=1)
df_police_calls['date'] = date_time[0]
df_police_calls['time'] = date_time[1]

AttributeError: 'DataFrame' object has no attribute 'str'

In [ ]:
# dropping useless columns
drop_cols_calls = ['address_dir_primary', 'address_dir_intersecting','incident_num']

df_police_calls = df_police_calls.drop(drop_cols_calls,axis = 1)

In [ ]:
# making everything lowercase
df_police_calls['address_road_primary']=df_police_calls['address_road_primary'].str.lower()
df_police_calls['address_road_intersecting']=df_police_calls['address_road_intersecting'].str.lower()
df_police_calls['address_sfx_primary'] = df_police_calls['address_sfx_primary'].str.lower()


In [ ]:
# strip any extra spaces in string data
df_police_calls['address_road_primary']= df_police_calls.address_road_primary.str.strip()
df_police_calls['address_sfx_primary'] = df_police_calls.address_sfx_primary.str.strip()
df_police_calls['address_road_intersecting']= df_police_calls.address_road_intersecting.str.strip()
df_police_calls['call_type'] = df_police_calls.call_type.str.strip()
df_police_calls['disposition'] = df_police_calls.disposition.str.strip()


In [ ]:
# convert the 'Date' column to datetime format 

df_police_calls['date']= pd.to_datetime(df_police_calls['date']) 
df_police_calls['time']= pd.to_datetime(df_police_calls['time']) 

In [ ]:
df_police_calls.dtypes

## df_police_stops

In [ ]:
drop_cols_stops = ['isstudent', 'gend_nc','agency','stop_id']
df_police_stops = df_police_stops.drop(drop_cols_stops, axis = 1)

In [ ]:
# making everything lowercase

df_police_stops['address_city'] = df_police_stops['address_city'].str.lower()


We can probably merge **address_road_primary** and **address_sfx_primary**

In [ ]:
df_police_stops.dtypes

In [ ]:
df_collisions.head()

visualization tools: https://medium.com/@stallonejacob/d3-in-juypter-notebook-685d6dca75c8

## Geospatial tools
- Geojson: https://pypi.org/project/geojson/
    - https://www.datacamp.com/community/tutorials/geospatial-data-python
- GeoPandas: http://geopandas.org/